<a href="https://colab.research.google.com/github/Gongsoo/Python3/blob/main/study/paper/papering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/paper"

/content/drive/MyDrive/paper


In [3]:
#!unzip -qq "/content/drive/MyDrive/paper/papering.zip"

In [4]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
CFG = {
    'IMG_SIZE':512,
    'EPOCHS':25,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

In [7]:
def seed_everything(seed):
    deterministic = False
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    if deterministic :
      torch.backends.cudnn.deterministic = True
      torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [8]:
all_img_list = glob.glob('/content/drive/MyDrive/paper/train/*/*')

In [9]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])

In [10]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])

In [11]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [12]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms = None,transforms_no_rotate=None, trainsforms_rotate = None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        self.transforms_no_rotate = transforms_no_rotate
        self.transforms_rotate = trainsforms_rotate
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        rotate_label = [1,2,4,6,7,8,9,10,11,12,15,16,17,18]
        image = cv2.imread(img_path)

        if self.label_list is not None:
            label = self.label_list[index]

            if self.transforms_rotate is not None and label in rotate_label :
                image = self.transforms_rotate(image=image)['image']
            elif self.transforms_no_rotate is not None and label not in rotate_label :
                image = self.transforms_no_rotate(image=image)['image']
            elif self.transforms_rotate is None and self.transforms_no_rotate is None and self.transforms is not None:
                image = self.transforms(image=image)['image']

            return image, label
        else:
            if self.transforms is not None :
              image = self.transforms(image=image)['image']
            return image
        
    def __len__(self):
        return len(self.img_path_list)


In [13]:
def calculate_norm(dataset,test):
    if test :
      mean_ = np.array([np.mean(x.numpy(), axis=(1, 2)) for x in tqdm(dataset)])
    else :
      mean_ = np.array([np.mean(x.numpy(), axis=(1, 2)) for x, _ in tqdm(dataset)])
    # r, g, b 채널에 대한 각각의 평균 산출
    mean_r = mean_[:, 0].mean()/255.
    mean_g = mean_[:, 1].mean()/255.
    mean_b = mean_[:, 2].mean()/255.

    if test :
      std_ = np.array([np.std(x.numpy(), axis=(1, 2)) for x in tqdm(dataset)])
    else :
      std_ = np.array([np.std(x.numpy(), axis=(1, 2)) for x, _ in tqdm(dataset)])
    # r, g, b 채널에 대한 각각의 표준편차 산출
    std_r = std_[:, 0].mean()/255.
    std_g = std_[:, 1].mean()/255.
    std_b = std_[:, 2].mean()/255.
    
    return (mean_r, mean_g, mean_b), (std_r, std_g, std_b)
     

In [14]:
train_resize = A.Compose([
                            ToTensorV2()
                            ])

test_resize = A.Compose([
                            ToTensorV2()
                            ])

In [15]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_resize,None,None)
val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_resize,None,None)

In [16]:
mean_train, std_train = calculate_norm(train_dataset,False)

  0%|          | 0/2765 [00:00<?, ?it/s]

  0%|          | 0/2765 [00:00<?, ?it/s]

In [17]:
mean_val, std_val = calculate_norm(val_dataset,False)

  0%|          | 0/692 [00:00<?, ?it/s]

  0%|          | 0/692 [00:00<?, ?it/s]

In [18]:
train_rotate_transform = A.Compose([
                            A.Normalize(mean_train, std_train,max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(p=0.5),
                            #A.RandomBrightness (limit=(0,0.1), always_apply=False, p=0.5),
                            A.OneOf([
                                A.Rotate(limit=30,interpolation=1, border_mode=4, value=None, mask_value=None, crop_border=False, always_apply=False, p=1),
                                A.RandomResizedCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE'],scale=(0.65,0.8), ratio=(0.65,0.8) ,p=1)
                            ], p=1),
                            A.OneOf([
                                A.MotionBlur(blur_limit=(3,4), p=1),
                                A.GaussianBlur (blur_limit=(3, 5), sigma_limit=0, always_apply=False, p=1)
                            ],p=1),
                            ToTensorV2()
                            ])

train_transform = A.Compose([
                            A.Normalize(mean_train, std_train,max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(p=0.5),
                            #A.RandomBrightness (limit=(0,0.1), always_apply=False, p=0.5),
                            A.OneOf([
                                A.ShiftScaleRotate(shift_limit = 0.1, scale_limit=0.1, rotate_limit=0,p=0.5),
                                A.RandomResizedCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE'],scale=(0.65,0.8), ratio=(0.65,0.8) ,p=1)
                            ], p=1),
                            A.OneOf([
                                A.MotionBlur(blur_limit=(3,7), p=1),
                                A.GaussianBlur (blur_limit=(3, 7), sigma_limit=0.5, always_apply=False, p=1)
                            ],p=1),
                            ToTensorV2()
                            ])

val_transform = A.Compose([
                            A.Normalize(mean_train, std_train,max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            ToTensorV2()
                            ])

In [21]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values,None, train_transform, train_rotate_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, val_transform,None,None)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

In [22]:
class Network(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(Network, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        x = F.log_softmax(x, dim=1)
        return x

In [23]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1,CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [24]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [25]:
model = Network()

optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.33, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 180MB/s]


  0%|          | 0/173 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.30274] Val Loss : [0.92268] Val Weighted F1 Score : [0.70674]


  0%|          | 0/173 [00:00<?, ?it/s]

...

  0%|          | 0/44 [00:00<?, ?it/s]

Epoch [24], Train Loss : [0.02769] Val Loss : [0.66532] Val Weighted F1 Score : [0.86120]


  0%|          | 0/173 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

Epoch [25], Train Loss : [0.02592] Val Loss : [0.70871] Val Weighted F1 Score : [0.86831]


In [27]:
torch.save(model, f'/content/drive/MyDrive/paper/no_bright_aug_0420_epoch{CFG["EPOCHS"]}_batch{CFG["BATCH_SIZE"]}_size{CFG["IMG_SIZE"]}_model.pt')

In [26]:
test = pd.read_csv('./test.csv')

In [28]:
test_dataset = CustomDataset(test['img_path'].values, None, test_resize,None,None)

In [29]:
mean_test, std_test = calculate_norm(test_dataset,True)

  0%|          | 0/792 [00:00<?, ?it/s]

  0%|          | 0/792 [00:00<?, ?it/s]

In [30]:
test_transform = A.Compose([
                            A.Normalize(mean_test, std_test, max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            ToTensorV2()
                            ])

In [31]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform,None,None)

In [32]:
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [33]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    return preds

In [39]:
preds = inference(model, test_loader, device)

  0%|          | 0/50 [00:00<?, ?it/s]

In [40]:
submit = pd.read_csv('./sample_submission.csv')

In [41]:
submit['label'] = preds

In [42]:
submit.loc[submit['label'] == 0, 'label'] = '가구수정'
submit.loc[submit['label'] == 1, 'label'] = '걸레받이수정'
submit.loc[submit['label'] == 2, 'label'] = '곰팡이'
submit.loc[submit['label'] == 3, 'label'] = '꼬임'
submit.loc[submit['label'] == 4, 'label'] = '녹오염'
submit.loc[submit['label'] == 5, 'label'] = '들뜸'
submit.loc[submit['label'] == 6, 'label'] = '면불량'
submit.loc[submit['label'] == 7, 'label'] = '몰딩수정'
submit.loc[submit['label'] == 8, 'label'] = '반점'
submit.loc[submit['label'] == 9, 'label'] = '석고수정'
submit.loc[submit['label'] == 10, 'label'] = '오염'
submit.loc[submit['label'] == 11, 'label'] = '오타공'
submit.loc[submit['label'] == 12, 'label'] = '울음'
submit.loc[submit['label'] == 13, 'label'] = '이음부불량'
submit.loc[submit['label'] == 14, 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == 15, 'label'] = '터짐'
submit.loc[submit['label'] == 16, 'label'] = '틈새과다'
submit.loc[submit['label'] == 17, 'label'] = '피스'
submit.loc[submit['label'] == 18, 'label'] = '훼손'

In [43]:
submit.to_csv(f'./no_bright_aug_0420_epoch{CFG["EPOCHS"]}_batch{CFG["BATCH_SIZE"]}_size{CFG["IMG_SIZE"]}_submitv2.csv', index=False)